In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [4]:
import pandas as pd


train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
sample_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")


print("Train data:")
display(train_df.head())

print("Test data:")
display(test_df.head())

print("Sample submission:")
display(sample_df.head())
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Train columns:", train_df.columns.tolist())
print("\nLabel distribution:")
print(train_df["winner_model_a"].value_counts(normalize=True))


Train data:


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


Test data:


,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


Sample submission:


,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.333333,0.333333,0.333333
1,211333,0.333333,0.333333,0.333333
2,1233961,0.333333,0.333333,0.333333


Train shape: (57477, 9)
Test shape: (3, 4)
Train columns: ['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie']

Label distribution:
winner_model_a
0    0.650921
1    0.349079
Name: proportion, dtype: float64


In [5]:
train_df["len_a"]=train_df["response_a"].str.len()

In [6]:
train_df["len_b"]=train_df["response_b"].str.len()

In [7]:
train_df[["len_a","len_b"]].describe()

,len_a,len_b
count,57477.000000,57477.000000
mean,1377.818919,1386.162169
std,1513.932333,1537.800488
min,4.000000,4.000000
25%,408.000000,413.000000
50%,1076.000000,1086.000000
75%,1862.000000,1873.000000
max,54058.000000,53830.000000


In [10]:
import pandas as pd
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test  = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
sample = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")
train.shape, test.shape, sample.shape


((57477, 9), (3, 4), (3, 4))

In [11]:
display(train.head)


<bound method NDFrame.head of                id             model_a              model_b  \
0           30192  gpt-4-1106-preview           gpt-4-0613   
1           53567           koala-13b           gpt-4-0613   
2           65089  gpt-3.5-turbo-0613       mistral-medium   
3           96401    llama-2-13b-chat  mistral-7b-instruct   
4          198779           koala-13b   gpt-3.5-turbo-0314   
...           ...                 ...                  ...   
57472  4294656694          gpt-4-0613             claude-1   
57473  4294692063          claude-2.0     llama-2-13b-chat   
57474  4294710549            claude-1           alpaca-13b   
57475  4294899228              palm-2       tulu-2-dpo-70b   
57476  4294947231  gemini-pro-dev-api   gpt-4-1106-preview   

                                                  prompt  \
0      ["Is it morally right to try to have a certain...   
1      ["What is the difference between marriage lice...   
2      ["explain function calling. how would 

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [13]:
train['winner_model_a'].value_counts(normalize=True)

winner_model_a
0    0.650921
1    0.349079
Name: proportion, dtype: float64

In [14]:
train['len_prompt']=train['prompt'].str.len()

In [15]:
train['len_a']=train['response_a'].str.len()

In [16]:
train['len_b']=train['response_b'].str.len()

In [17]:
train[['len_prompt','len_a','len_b']].describe()

,len_prompt,len_a,len_b
count,57477.000000,57477.000000,57477.000000
mean,369.198584,1377.818919,1386.162169
std,1073.341472,1513.932333,1537.800488
min,7.000000,4.000000,4.000000
25%,52.000000,408.000000,413.000000
50%,96.000000,1076.000000,1086.000000
75%,243.000000,1862.000000,1873.000000
max,33056.000000,54058.000000,53830.000000


In [19]:
train["text"] = train["prompt"].fillna('') + " [SEP] " + train["response_a"].fillna('') + " [SEP] " + train["response_b"].fillna('')
test["text"]  = test["prompt"].fillna('') + " [SEP] " + test["response_a"].fillna('') + " [SEP] " + test["response_b"].fillna('')


In [20]:
print(train.columns)
print(test.columns)


Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie', 'len_prompt', 'len_a',
       'len_b', 'text'],
      dtype='object')
Index(['id', 'prompt', 'response_a', 'response_b', 'text'], dtype='object')


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer   # ✅ Correct class name

tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train["text"])
X_test = tfidf.transform(test["text"])

y_train = train["winner_model_a"]   # ✅ Correct column name


In [24]:
y_train

0        1
1        0
2        0
3        1
4        0
        ..
57472    1
57473    1
57474    1
57475    0
57476    1
Name: winner_model_a, Length: 57477, dtype: int64

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
model=LogisticRegression(max_iter=200)
model.fit(X_train,y_train)

LogisticRegression(max_iter=200)

In [27]:
pred_probs=model.predict_proba(X_test)[:,1]

In [28]:
pred_probs

array([0.1725572 , 0.30735739, 0.38534224])

In [30]:
submission=pd.DataFrame({
    "id":test["id"],
    "winner_model_a":pred_probs
})

In [31]:
submission.to_csv("/kaggle/working/submission.csv",index=False)

In [32]:
submission.head()

,id,winner_model_a
0,136060,0.172557
1,211333,0.307357
2,1233961,0.385342


In [33]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model.fit(X_tr, y_tr)
pred_val = model.predict_proba(X_val)[:, 1]

print("Log Loss:", log_loss(y_val, pred_val))
print("Accuracy:", accuracy_score(y_val, (pred_val>0.5).astype(int)))


Log Loss: 0.6579401799564939
Accuracy: 0.6370911621433543
